**Group Members:** Tony Liang (004), Wanxin luo (003), Xuan Chen (004)

**Student Numbers:** 39356993, 33432808, 15734643


ECON 323 Quantitative Economic Modelling with Data Science Applications UBC 2023

# Boston Housing Price Prediction Proposal

In [1]:
# Imports of libraries

# essential utilities
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
# Models (linear and tree)
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
# custom function imports
from src.handle_data import get_data, split_data, merge_data
from src.modelling import mean_std_cross_val_scores, fit_model, get_metrics
from src.plotting import eda_plot

## Introduction

In this project, we aim to **explore the impact of environmental factors on housing prices** using the [Boston Housing dataset](https://www.kaggle.com/datasets/fedesoriano/the-boston-houseprice-data.). This dataset contains information on various attributes such as crime rate, average number of rooms, accessibility to highways, and more, which are hypothesized to influence housing prices. The project will involve several parts, including data cleaning, visualization, and model building. Our objective is to conduct exploratory data analysis (EDA) and then build a hedonic regression model with multiple inputs. We will utilize various Python techniques learned in this course to explore the real world data and solve economic questions.

By analyzing the data, we aim to answer economic questions related to the housing market and explore the real-world application of Python techniques. It is important to note that the dataset has its limitations as it was collected almost 50 years ago, but it still provides an excellent opportunity for us to apply our Python skills and gain insights of housing market.

> Original
<!--- Original--->
---
<!--- Edit from Sherry--->

Boston, a thriving metropolitan city known for its rich history and cultural significance, has grappled with the challenge of crime rateand its potential impact on housing prices. Based on past data, the crime rate in Boston is nearly double the national crime rate.In this research, we aim to shed light on the relationship between crime rate and housing prices in Boston, drawing on a hedonic regression model to quantitatively analyze data. Drawing on a comprehensive set of variables, including not only structural characteristics of the housing but also hedonic variables such as crime occurrences in the neighborhood, toursim factor as Charles River and lower status of the population, we seek to provide insights into how crime impacts housing prices in Boston, and to what extent it influences the dynamics of the local real estate market. Our hypothesis aim to show that the **increasing criminal rate in Boston is a significant factor in the determination of housing market prices**.

> Sherry edit


### Dataset Description

The Boston Housing Dataset is a derived from information collected by the U.S. Census Service concerning housing in the area of Boston, MA. The following describes the dataset columns:

- `CRIM` - per capita crime rate by town
- `ZN` - proportion of residential land zoned for lots over 25,000 sq.ft.
- `INDUS` - proportion of non-retail business acres per town.
- `CHAS` - Charles River dummy variable (1 if tract bounds river; 0 otherwise)
- `NOX` - nitric oxides concentration (parts per 10 million)
- `RM` - average number of rooms per dwelling
- `AGE` - proportion of owner-occupied units built prior to 1940
- `DIS` - weighted distances to five Boston employment centres
- `RAD` - index of accessibility to radial highways
- `TAX` - full-value property-tax rate per $10,000$
- `PTRATIO` - pupil-teacher ratio by town
- $B - 1000(Bk - 0.63)^2$ where $B_k$ is the proportion of blacks by town
- `LSTAT` - $%$ lower status of the population
- `MEDV` - Median value of owner-occupied homes in $1000$'s

The dataset is derived from https://www.kaggle.com/datasets/fedesoriano/the-boston-houseprice-data.

> Original
<!--- Original --->
---
<!--- Edit --->
Housing is a heterogenous good, which comprises a collection of attributes, and its price is determined by the implicit prices of those attributes. The attributes that determine the price of housing can be broadly classified into three groups: site characteristics, neighborhood characteristics, and environmental characteristics. 
- The site characteristics include： `ZN`、`RM`、`AGE`、`TAX`、`MEDV`
- The neighborhood characteristics include：`CRIM`、`INDUS`、`CHAS`、`DIS`、`RAD` 、`PTRATIO`、`LSTAT`、`B`
- The environmental characteristics include：`NOX`

> Sherry edit

## Methods

This report strives to be trustworthy using the following steps: 

1. [Thorough EDA](#eda)
2. [Model fitting](#model-fitting)
3. [Model selection](#model-selection)

**Note**: this could be subjected to changes later after feedback from the ECON323 Instrutor's team


### EDA

During the EDA phrase, we will conduct a thorough examination of the Boston Housing dataset. One of the key steps is to generate a correlation matrix, which can help us identify any potential issues related to multicollinearity between the independent variables. In addition, we will use side-by-side box plots to visualize the distributions of the continuous variables and detect any potential outliers or anomalies. Moreover, we will leverage other data visualization techniques, such as scatter plots and histograms, to better understand the relationships between the variables and explore potential trends or patterns in the data. Overall, the goal of EDA is to gain insights into the data and inform our subsequent modelling steps. 

In [2]:
# read in the dataset
data_path = 'data/boston_housing_data.csv'
boston = get_data(data_path=data_path)
# check the first rows of the dataset
boston.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,NaN,36.2


Upon observing the first 5 rows of the data, we can identify the categorical and continuous variables. Although most of features are numeric variables, there are still some special variables. For instance, the `CHAS` variable is a dummy variable indicating whether the tract bounds the Charles River or not, and it is encoded as 0 or 1. Additionally, the `RAD` variable is an indexed variable that may be dropped in the future analysis.

In [3]:
# # summarize the dataset 
# boston.describe()

According to the summary table above, we can see the basic statistical information for numerical data, including count, mean, standard deviation, minimum, 25% percentile, median, 75% percentile, and maximum. We can see that the count of each column is different, which means we have missing values here. Therefore, we will check and handle the missing values by using imputation while fitting models.

Certainly, there is one column in the dataset that requires special attention, which is the `CHAS` column. `CHAS` is a dummy variable that takes a value of 1 if the property is adjacent to the Charles River, and 0 otherwise. The `CHAS` are 0 for the 25th, 50th, and 75th percentiles, indicating that the distribution of this variable is skewed. To gain a deeper understanding of the data and identify any interesting trends or statistics, it may be beneficial to visualize the dataset through plotting.

In [4]:
# eda_plot(data = boston, title = 'Figure 1: Boxplot of Boston Housing Features', option = "box")

According to the Figure 1 above, we can observe that several columns in the Boston dataset such as `CRIM`, `ZN`, `RM`, and `B` appear to have many outliers. In the context of machine learning, extreme outliers can be problematic because they can skew statistical measures such as mean and standard deviation. 

In [5]:
# eda_plot(data = boston, title = 'Figure 2: Histogram of Boston Housing Features', option = "hist")

In addition to identifying outliers, it is also important to understand the distributions of the individual columns in the dataset. The histograms generated earlier suggest that columns `CRIM`, `ZN`, `LSTAT`, `DIS` and `B` have skewed distributions. 
The distributions of the other columns appear to be normal or bimodal, except for `CHAS`, which is a discrete variable.
In contrast, the histogram of `MEDV` (the variable we are trying to predict) appears to have a roughly normal distribution.

In [6]:
# eda_plot(data = boston, title = 'Figure 3: Correlation Matrix Heatmap for Boston Dataset', option = "heatmap")

Based on the Figure 3 above, it can be observed that there are several variables that exhibit a strong correlation with the target variable, `MEDV`. Specifically, the variable `RM` exhibits a correlation coefficient of $0.7$ with `MEDV`, while `LSTAT` exhibits a correlation coefficient of $0.74$ with `MEDV`. These correlations are relatively high compared to the other variables, and suggest that `RM` and `LSTAT` may be useful predictors for our regression model.

However, it is also important to note that the variable `TAX` exhibits a strong correlation with the variable `RAD`, with a correlation coefficient of $0.91$. This suggests the presence of multicollinearity, which can cause issues in regression analysis such as inflated standard errors, reduced statistical power, and unstable coefficients. Therefore, it may be necessary to address multicollinearity in the modeling process, perhaps deleting one of the predictors.

> I honestly think below is redundant 

In [7]:
# import seaborn as sns #RM has the highest correlaton
# import matplotlib.pyplot as plt
# bos_df_univariate = boston[['RM', "MEDV"]]
# sns.pairplot(bos_df_univariate)

### Model Fitting

In the model fitting phase, we will split the Boston Housing dataset into training and testing sets. We will then use the training set to select the relevant variables and build our final multiple linear regression model. The selection process can involve various techniques, such as stepwise regression or regularization, depending on the specific requirements of the project. Once we have the final model, we will use the testing set to evaluate its performance in terms of mean squared error (`MSE`). The goal is to ensure that the model can generalize well to new, unseen data and make accurate predictions. 

To further explore effects of using different methods of regression, we are going to fit multiple models to using similar metrics accross these to compare best fit of model, i.e. Root Mean Squared Error (RMSE) and Mean Absolute Error (MAE) for prediction purposes models. Our ideal approach to test this is to fit the following models:
1. Ordinary Least Squares (OLS) as baseline 
2. OLS with L1-norm regularization (Least Absolute Shrinkage and Selection Operator (LASSO Regression) )
3. Random Forest Regression

> Note: We are only interested in making predictions only, so this part of **Model Fitting** might need to refactor a bit
>
> Essentially get rid of Adjusted $R^2$ and BIC stuffs, we should only keep RMSE, MSE, MAE (could also be optional, but need to let me know beforehand to edit code correspondingly)

In order to compare different machine learning models, we will use the same scoring metrics(defined below) across all models. We will first perform a raw fit to all models to see how they perform against the test set. This step will give us a baseline performance of each model before applying any preprocessing or hyperparameter optimization.

Next, we will apply common preprocessing steps to all models, such as scaling or imputation. After applying the preprocessing steps, we will optimize the hyperparameters for each model using 5-fold cross-validation. This step will help us identify the best combination of hyperparameters for each model.

Finally, we will compare the performance of all models again against the test set by using the same scoring metrics. This comparison will help us identify the most suitable model for our prediction problem.

> This is just an outline of what to do, It is just like a reminder, and could be better phrased. 

> Update: already phrased by Wanxin Luo

#### Load and Split data

To begin, we will split the data into train and test data for further modelling purposes. The training set will be used to train the three models and find the best hyperparameters. The testing set will be used to evaluate each model's performance on previously unseen data, which helps us to assess models' ability to generalize beyond the training set. Update：浅改了一下。

In [8]:
# use same random_state across all parts to ensure reproducibility
random_state = 20230325

# Splits the data into X and y train and test portions
X_train, X_test, y_train, y_test = split_data(data_path, proportion = 0.75, target = "MEDV", random_state=random_state)

Moreover, we are going to use common scoring metrics (as mentioned earlier) across all models, so that we could compare their effectiveness in prediction:
- Root Mean Squared Error (RMSE)
- Mean Absolute Percentange Error (MAPE)
- Mean Squared Error (MSE)

In [9]:
# define common scoring across models
scoring = {
    "neg_rmse": "neg_root_mean_squared_error",
    "neg_mape": "neg_mean_absolute_percentage_error", 
    "neg_mse": "neg_mean_squared_error",
}

We could fit all the previous models mentioned at once and perform cross validation, in a more rapid way to compare and check which might perform better at **predicting house prices**.
Then we could fit the methods individually in sections below raw, then with preprocessing, and check if getting similar scores like these CVs.

In [10]:
# # fit all models at once to decide which one to use
# # User-defined scoring to be hanlde in-terms of calculating scores for model
# # Note scikit learn implemented these scorings in negative scale to minimize loss
# # which we will scale back to positive behind the scenes

def fit_all(scoring=scoring, **kwargs):
    methods = ["OLS", "LASSO", "Random Forest"]
    models = [mod for mod in [LinearRegression(), Lasso(), RandomForestRegressor()]]
    # allocate space to store model fitted outputss
    result_dict = {}
    # fit all of these models at once
    for method, model in zip(methods, models):
        result_dict[method] = mean_std_cross_val_scores(model, 
                                                       X_train, 
                                                       y_train, 
                                                       return_train_score=True,
                                                       scoring=scoring
                                                     )
    
    result = pd.DataFrame(result_dict)
    # rename the index to positive scale for easier comparison, replace the neg by empty
    return result.rename(lambda x: x.replace("_neg", ""))
all_cv = fit_all(X_train=X_train, y_train=y_train)
all_cv.style.set_caption("Means CVs and +- std for Regression Methods")

,OLS,LASSO,Random Forest
fit_time,0.003 (+/- 0.006),0.000 (+/- 0.000),0.179 (+/- 0.011)
score_time,0.000 (+/- 0.000),0.003 (+/- 0.006),0.009 (+/- 0.007)
test_rmse,4.520 (+/- 0.788),5.003 (+/- 0.585),3.387 (+/- 0.442)
train_rmse,4.278 (+/- 0.197),4.858 (+/- 0.167),1.332 (+/- 0.130)
test_mape,0.163 (+/- 0.020),0.177 (+/- 0.023),0.125 (+/- 0.014)
train_mape,0.152 (+/- 0.006),0.168 (+/- 0.004),0.044 (+/- 0.001)
test_mse,20.925 (+/- 7.115),25.308 (+/- 5.804),11.625 (+/- 2.868)
train_mse,18.329 (+/- 1.684),23.627 (+/- 1.630),1.789 (+/- 0.339)


From above table, we could see fit time, score time, the scorings we set earlier **(MAPE, RMSE, MSE)**. And, each entry is the mean cross-validation (cv) scores $\pm$ the standard deviation. 

Based on the test scores, the random forest (RF) model performs the best, this is expected since trees could capture non-linear relationships (if existed) between variables. Also, it trains multiple decision trees simultaneously and reduces variance through bagging or bootstrap over random subsets of variables and observations.  

For linear models, we observed that plain Ordinary Least Squares (OLS) performs better than LASSO. This could be due to the fact that we only used the default value for the regularization parameter $\alpha$ in LASSO, which might not have selected enough variables. Furthermore, OLS by definition minimizes the prediction error, which is why it outperforms LASSO with default parameters.

For more information, we could also look at training scores, the tree model has the lowest **train** score, which is also expected, since trees are tended to overfit if we set the depth to infinity or if not constrained. However, it takes longer fitting time due to its nature of bagging an ensemble of multiple regression trees. 

> I wrote this part by plain memory, some stuffs might not be accurate or true. Just like a general idea of what interpreation could be provide here @Tony

> "And it reduces variance (?) by bagging or bootstrap over random subsets of variables and observations." This is right, I the rephrased the entire paragraph a little bit. @ Wanxin

Based on the preliminary test scores, we might have some sense of which models might work better for predicting housing prices in Boston. We hypothesize that RF might the best model, and OLS would serve as our baseline model to compare. Moving forward, we will fit all models on both raw and preprocessed data with optimized hyperparameters to determine the best model for predicting housing prices in Boston. Of course, OLS would serve as our baseline model to compare. 

> 改了一点by Wanxin

#### Ordinary Least Squares (OLS)
In the first stage of modeling, we will apply the traditional hedonic ordinary least squares regression to determine the coefficients between housing characteristics and housing prices. With the OLS approach, we can estimate the impact of each attribute on the market value of a heterogeneous good, and then we estimate prices in the form of willingness to pay for one unit of change of that characteristic.

Now, we are going to fit a plain OLS regression model to act as our baseline model for comparing with other regression methods and see their improvements or weakness when applying regularization or boosting and bootstrapping. We propose a mixed model with different exponents for the explanatory variables as following form:

$$P = \beta_0 + \beta_1 \text{CRIM} + \beta_2 \text{ZN} +\beta_3 \text{INDUS}+\beta_4 \text{CHAS}+\beta_5 \text{NOX}+\beta_6 \text{RM}+\beta_7 \text{AGE}+\beta_8 \text{DIS}+\beta_9 \text{RAD}+\beta_{10} \text{TAX}+\beta_{11} \text{PTRATIO}+\beta_{12} \text{B}+\beta_{13} \text{LSTAT}Price of housing unit in Boston$$

whereas $P$ is the dependent variable `Price of housing unit in Boston`, and other $\beta_i$ are beta estimates of independent variables $x_i \quad \forall i \in [0, 13]$, and $\beta_0$ is a special case, such it is the value of estimated $y$, where all the independent variables equal to 0.

Hence, above equation can be generalized into matrix form below:

$$Y = X\beta + \epsilon$$

where $X$ is the design matrix with leading column of 1s (to represent the intercept term) and columns of independent variables $x_1, \dots, x_{14}$, $\beta$ is the matrix of all estimates from $\beta_0, \dots, \beta_{14}$, and $\epsilon$ is the  random error of measurements

Then solving for $\beta$ yields to the following:

$$\beta = (X^{T}X)^{-1}X^{T}Y$$

Hence, we could use this above to solve for our regression.

In [15]:
# call custom helper to fit OLS model and predict on test data, based on selected metrics
raw_lr = fit_model(X_train=X_train, y_train=y_train, X_test=X_test, y_test=y_test, name="OLS")
raw_lr_scores = get_metrics(predicted=raw_lr.predict(X_test), actual=y_test, name="OLS")
# preprocess train and use that to train model and predict on test
lr_tuned = fit_model(X_train=X_train, y_train=y_train, X_test=X_test, y_test=y_test, name="OLS", preprocess=True)
lr_tuned_scores = get_metrics(predicted=lr_tuned.predict(X_test), actual=y_test, name="OLS Predicted")
# merge results and show it
lr_mods = merge_data(raw_lr_scores, lr_tuned_scores)
lr_mods

ValueError: No valid specification of the columns. Only a scalar, list or slice of all integers or all strings, or boolean mask is allowed

>  * RMSE is the square root of Mean Squared error and measure the standard deviation of residuals,which tells us the typical distance between the predicted value made by the regression model and the actual value. In the "OLS" model, the RMSE is 4.954, while in the "OLS + preprocessed" model, the RMSE is slightly lower at 4.922. This suggests that the "OLS + preprocessed" model may have slightly better predictive performance compared to the "OLS" model.
> * MSE is average of the squared difference between the original and predicted values in the data set. It measures the variance of the residuals. In the "OLS" model, the MSE is 24.539, while in the "OLS + preprocessed" model, the MSE is slightly lower at 24.224. This further supports the observation that the "OLS + preprocessed" model may have slightly better predictive performance compared to the "OLS" model.
> * MAPE is represents the average of the absolute difference between the actual and predicted values in the dataset. It measures the average of the residuals. In the "OLS" model, the MAPE is 0.161, while in the "OLS + preprocessed" model, the MAPE is lower at 0.152. This suggests that the "OLS + preprocessed" model may have better predictive performance in terms of percentage errors compared to the "OLS" model.

#### Multiple Regression

In [ ]:
# import statsmodels.api as sm

# X = X_train[['CRIM', 'ZN', 'INDUS','CHAS','NOX','RM','AGE','DIS','RAD','TAX','PTRATIO','B','LSTAT']]
# y = y_train

# #add constant to predictor variables
# X = sm.add_constant(X)

# #fit linear regression model
# model = sm.OLS(y, X).fit()

# #store parameter estimates
# beta_0_multi = model.params['const']
# beta_1_multi = model.params['CRIM']
# beta_2_multi = model.params['ZN']
# beta_3_multi = model.params['INDUS']
# beta_4_multi = model.params['CHAS']
# beta_5_multi = model.params['NOX']
# beta_6_multi = model.params['RM']
# beta_7_multi = model.params['AGE']
# beta_8_multi = model.params['DIS']
# beta_9_multi = model.params['RAD']
# beta_10_multi = model.params['TAX']
# beta_11_multi = model.params['PTRATIO']
# beta_12_multi = model.params['B']
# beta_13_multi = model.params['LSTAT']

# #view fitted model
# print(f"Price of housing price= {beta_0_multi}+ {beta_1_multi:.4f} CRIM + {beta_2_multi:.4f}ZN+{beta_3_multi:.4f} INDUS+{beta_4_multi:.4f} CHAS+{beta_5_multi:.4f} NOX + {beta_6_multi:.4f} RM + {beta_7_multi:.4f} AGE+{beta_8_multi:.4f} DIS+{beta_9_multi:.4f} RAD + {beta_10_multi:.4f} TAX +{beta_11_multi:.4f} PTRATIO+{beta_12_multi:.4f} B+{beta_13_multi:.4f} LSTAT")

In [ ]:
# print(model.summary())

> Add interpretation here

> * Interpretation of intercept:The intercept gives us the average housing price in boston,which is 38.8110 in this sample.
> * Interpretation of each regressors: For CRIM: the coefficient for CRIM is -0.1095, which means that for every one-unit increase in CRIM, the estimated change in MEDV is a decrease of 0.1095 units, holding all other variables constant.
> * Using the F statistics to determine the significantly difference: Our F-statistic result is 64.30, with a very low p-value of 2.45e-76, which indicating that  we have strong evidence against the null hypothesis that all regression coefficients are zero and then the regression model is statistically significant.
> * In this fitted model, above table shows the estimated coefficients for each explantory variables. The R-squared means that approximately 74.8% of total valuation is explained by the regression model and we obtained a value for the adjusted determination coefficient of 0.736, which indicates that we have a good fit. Additionally, most of the variables are significant, and they have the expected sign.


* For this part,we want to use Variance Inflation Factor to check the Multicollinearity,if there are any high correlations among the predictor variables, then we proceeded to eliminate it. 

In [ ]:
# from statsmodels.stats.outliers_influence import variance_inflation_factor
# from sklearn.model_selection import cross_val_score

# # the independent variables set
# X2 = X_train[['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT']]
  
# # VIF dataframe
# vif2_data = pd.DataFrame()
# vif2_data["Feature"] = X2.columns
  
# # calculating VIF for each feature
# vif2_data["VIF"] = [variance_inflation_factor(X2.values, i)
#                           for i in range(len(X2.columns))]
  
# print(vif2_data)

* !!! Yes,we have super high VIF at here, we need to drop until the each feature's VIF is less than 10.
* firstly, we drop the TAX,PTRATIO,NOX,RM,B,Age because those are extremely big.

In [ ]:
# # the independent variables set
# X2 =  X_train[['CRIM', 'ZN', 'INDUS', 'CHAS', 'DIS', 'RAD', 'LSTAT']]
  
# # VIF dataframe
# vif2_data = pd.DataFrame()
# vif2_data["Feature"] = X2.columns
  
# # calculating VIF for each feature
# vif2_data["VIF"] = [variance_inflation_factor(X2.values, i)
#                           for i in range(len(X2.columns))]

# print(vif2_data)

* Now,all of the variable is less than 10! We deducts variables who has too much correlation between the variable Xj and the other explanatory variables. 
* We construct a new model with no problem of multicollinearity according to the VIF analysis since all the variables in this model had a VIF under 10.

In [ ]:
# X = X_train[['CRIM', 'ZN', 'INDUS','CHAS','DIS','RAD','LSTAT']]
# y = y_train

# #add constant to predictor variables
# X = sm.add_constant(X)

# #fit linear regression model
# model2 = sm.OLS(y, X).fit()

# #store parameter estimates
# beta_0_multi = model.params['const']
# beta_1_multi = model.params['CRIM']
# beta_2_multi = model.params['ZN']
# beta_3_multi = model.params['INDUS']
# beta_4_multi = model.params['CHAS']
# beta_5_multi = model.params['DIS']
# beta_6_multi = model.params['RAD']
# beta_7_multi = model.params['LSTAT']


# #view fitted model
# print(f"Price of housing price= {beta_0_multi}+ {beta_1_multi:.4f} CRIM + {beta_2_multi:.4f}ZN+{beta_3_multi:.4f} INDUS+{beta_4_multi:.4f} CHAS+{beta_5_multi:.4f} DIS + {beta_6_multi:.4f} RAD + {beta_7_multi:.4f} LSTAT")

In [ ]:
# lm = LinearRegression()
# lm.fit(X_train, y_train)

# y_pred = lm.predict(X_train)
# y_pred

# plt.figure(figsize = (10,5))
# sns.regplot(x=y_train,y=y_pred)
# plt.title('Linear regression with all features', fontsize = 20)

In [ ]:
# print(model2.summary())

* After the Homoscedasticity and nonautocorrelation were verified, while the assumption of normality was not met. To solve the problem of this distributional assumption, we do the following part to let we can obtain more accurate inferences

#### LASSO Regression

In [ ]:
# call custom helper to fit LASSO model and predict on test data, based on selected metrics
raw_lasso = fit_model(X_train=X_train, y_train=y_train, X_test=X_test, y_test=y_test, name="LASSO")
# preprocess train and use that to train model and predict on test
lasso_tuned = fit_model(X_train=X_train, y_train=y_train, X_test=X_test, y_test=y_test, name="LASSO", preprocess=True)
# merge results and show it
lasso_mods = merge_data(raw_lasso, lasso_tuned)
lasso_mods

> Add interpretaion here

#### Random Forest

Other than using linear models, we are so using tree-like models, particularly ensembles of multiple decision regression trees which is a Random Forest.

In [ ]:
# call custom helper to fit Random Forest model and predict on test data, based on selected metrics
raw_rf = fit_model(X_train=X_train, y_train=y_train, X_test=X_test, y_test=y_test, name="RF",
                   random_state=random_state)
# preprocess train and use that to train model and predict on test
rf_tuned = fit_model(X_train=X_train, y_train=y_train, X_test=X_test, y_test=y_test, name="RF", 
                     preprocess=True, random_state=random_state)
# merge results and show it
rf_mods = merge_data(raw_rf, rf_tuned)
rf_mods

> Add interpretaion here

In [ ]:
# show all results
all_mods = merge_data(lr_mods, lasso_mods, rf_mods)
# get the mean test cvs scores earlier to compare with the ones with got right now 
cvs = all_cv.loc[["test_rmse", "test_mse", "test_mape"]].rename(lambda x: x.replace("test_", "").upper()).add_suffix("_CV").T

In [ ]:
merge_data(cvs, all_mods)

> Add interpretaion here
>
> Note: I have not got to optimal hyperparameters of LASSO nor Random Forest yet

## Feature Selection

Based on our initial analysis, the random forest model doesn't demonstrate any significant improvement compared to other models in predicting the housing prices in Boston. Moreover, this model is relatively complex and difficult to interpret, which limits its practical application. Therefore, we decide to use the Lasso regression method to select relevant variables and then fit those selected variables into an OLS model to evaluate its performance on the test set.

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_transformer
from sklearn.model_selection import cross_val_score

preprocessor = preprocess_data(X_train)

best_score = 0

param_grid = {"alpha": 10.0 ** np.arange(-3, 6, 1)}

results_dict = {"alpha": [], "mean_cv_score": []}

for a in param_grid[
    "alpha"
]:  
    pipe = make_pipeline(preprocessor, Lasso(alpha=a))
    scores = cross_val_score(pipe, X_train, y_train)  # perform cross-validation
    mean_score = np.mean(scores)  # compute mean cross-validation accuracy
    if (
        mean_score > best_score
    ):  # if we got a better score, store the score and parameters
        best_score = mean_score
        best_params = {"alpha": a}
    results_dict["alpha"].append(a)
    results_dict["mean_cv_score"].append(mean_score)

In [ ]:
best_params

In [ ]:
boston.columns

In [ ]:
# Create a pipeline with preprocessing steps and Lasso regression
pipe_lasso = make_pipeline(preprocessor, Lasso(alpha=0.01))

# Fit the pipeline to the training data
pipe_lasso.fit(X_train, y_train)

# Extract the coefficients of the Lasso model
coeffs = pipe_lasso.named_steps["lasso"].coef_

# Create a DataFrame to display the coefficients
pd.DataFrame(coeffs, 
             index=['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'TAX',
                    'PTRATIO', 'B', 'LSTAT'], 
             columns=["Coefficients"])

need to check the performance in the test set, but it's redundant for me to do here, maybe adjust the code in the python scripts in the folder src is better.

In [ ]:
# source from https://scikit-learn.org/stable/auto_examples/ensemble/plot_forest_importances.html
# modified by Tony Liang
def show_feature_importances(mod, X=X_train, plot=False):
    # get the importance of variables 
    importances = mod.feature_importances_
    # get stan dard deviation of these variables across individual trees
    std = np.std([tree.feature_importances_ for tree in mod.estimators_], axis=0)
    # get names of our original variables
    feature_names = [col for col in X_train.columns]
    # wrap to pandas series
    forest_importances = pd.Series(importances, index=feature_names)
    
    if plot is True:
        # plot the feature importances
        fig, ax = plt.subplots()
        # add std line as xerr
        forest_importances.plot.barh(xerr=std, ax=ax)
        ax.set_title("Variable importances in predicting house price in Boston")
        ax.set_ylabel("Mean decrease in impurity")
        fig.tight_layout()
    else:
        out = forest_importances.to_frame(name="Importance").sort_values(by="Importance", ascending=False)
        return out

In [ ]:
# # show plot view of importances
# show_feature_importances(mod=optimal_rf, plot=True)
# # show dataframe view of importances
# importance_df = show_feature_importances(mod=optimal_rf)
# importance_df

## Division of Labor
Based on the previous discussions, the team has divided the responsibilities as follows:

- Tony: Coding
- Wanxin: Coding and some textual descriptions
- Xuan: Written section of the report

However, the team may make adjustments to the division of labor as needed during the project to ensure that all tasks are completed efficiently and effectively. Effective communication and collaboration within the team will be critical to ensure that everyone is working together towards the same goal.

## References

Vishal, V. (2017, October 27). Boston Housing Dataset. Kaggle. Retrieved March 14, 2023, from https://www.kaggle.com/datasets/altavish/boston-housing-dataset 

## Unused Code

<!-- According to the summary table above, we can see the basic statistical information for numerical data, including count, mean, standard deviation, minimum, 25% percentile, median, 75% percentile, and maximum. 

Certainly, there is one column in the dataset that requires special attention, which is the CHAS column. CHAS is a dummy variable that takes a value of 1 if the property is adjacent to the Charles River, and 0 otherwise. The CHAS are 0 for the 25th, 50th, and 75th percentiles, indicating that the distribution of this variable is skewed. To gain a deeper understanding of the data and identify any interesting trends or statistics, it may be beneficial to visualize the dataset through plotting.### Model Fitting

In the model fitting phase, we will split the Boston Housing dataset into training and testing sets. We will then use the training set to select the relevant variables and build our final multiple linear regression model. The selection process can involve various techniques, such as stepwise regression or regularization, depending on the specific requirements of the project. Once we have the final model, we will use the testing set to evaluate its performance in terms of mean squared error (`MSE`). The goal is to ensure that the model can generalize well to new, unseen data and make accurate predictions. 

To further explore effects of using different methods of regression, we are going to fit multiple models to using similar metrics accross these to compare best fit of model, i.e. Bayesian Information Criterion (BIC) and Adjusted $R^2$ for inference (how well our model explains the effects of the explanatory variables is of the variable of interes); Root Mean Squared Error (RMSE) and Mean Absolute Error (MAE) for prediction purposes models. Our ideal approach to test this is to fit the following models:
1. Ordinary Least Squares (OLS) as baseline 
2. OLS with L2-norm regularization (Ridge Regression)
3. OLS with L1-norm regularization (Least Absolute Shrinkage and Selection Operator (LASSO Regression) )
4. Decision Tree Regression
5. Random Forest Regression -->

In [ ]:
# get the intercept and beta estimates
# scikit-learn imported function
# model = LinearRegression().fit(X_train, y_train)
# self-defined function
# intercept, estimates = OLS(X_train, y_train)
# Use this option to NOT show scientific notation (default shows scientific notation)
# np.set_printoptions(suppress=True)
# # print the estimates from self-defined func
# print(f"From self defined OLS, the intercept is {round(intercept,3)}, \nAnd beta estimates are: \n{np.ndarray.round(estimates, decimals=3)}")

# # prints the estimates from scikit-learn to inspect and compare
# print(f"\nFrom scikit-learn function, the intercept is {round(model.intercept_, 3)}, \nAnd beta estimates are: ")
# print(np.ndarray.round(model.coef_[1:], decimals=3))